# Script to load conceptnet data with unique edges.

*Issue*: loading edges fails for all unique edges. check methods to resolve this.

## Peek into the data 

### ConceptNet - Word

In [33]:
import pandas as pd 

root = 'csv_imports/'

nodes = pd.read_csv(root+'WN-nodes.csv')

nodes.head(1)

,uri,id,word,pos,definition,subject
0,http://wordnet-rdf.princeton.edu/id/00001740-a,00001740-a,able,adjective,(usually followed by `to') having the necessar...,adj.all


### ConceptNet - Synset

In [34]:
synsets = pd.read_csv(root+'synsets.csv')
synsets.head(1)

,id:ID,pos:string,definition:string,:LABEL
0,able.a.01,a,(usually followed by `to') having the necessar...,Synset


In [35]:
synsets_enc = pd.read_csv(root+'encoded/synsets-encoded.csv',index_col=0)
synsets_enc.head(1)

#helper method to encode synsets relationship columns.
# def get_synset(id):
#     try:
#       s = id.split('.')
#       return s[0]
#     except:
#       return ""
        
# synsets_enc['name'] = synsets_enc.apply(lambda x: get_synset(x['id:ID']),axis=1)
# synsets_enc.to_csv('encoded/synsets-encoded.csv')

,pos:string,definition:string,:LABEL
id:ID,,,
able.a.01,0,(usually followed by `to') having the necessar...,Synset


### ConceptNet - Edges

In [36]:
relationships = pd.read_csv(root+'relationships.csv',index_col=[0])
relationships.head(5)

,:END_ID,dataset:string,weight:double,:TYPE
:START_ID,,,,
able.a.01,ability.n.01,/d/wordnet/3.1,2.0,Attribute
able.a.01,ability.n.02,/d/wordnet/3.1,2.0,Attribute
able.a,able.a.01,/d/wordnet/3.1,2.0,InSynset
unable.a.01,ability.n.01,/d/wordnet/3.1,2.0,Attribute
unable.a,unable.a.01,/d/wordnet/3.1,2.0,InSynset


### ConceptNet - Part of Speech

In [37]:
relationships = pd.read_csv(root+'encoded/pos_wn.csv',index_col=[0])
relationships.head()

,type,id
0,adjective,0
1,noun,3
2,adverb,2
3,verb,4
4,adjective_satellite,1


## Install pyTigerGraph

In [38]:
!pip install -U pyTigerGraph

## Add Imports and Establish Initial Connection

In [39]:
# Imports
import pyTigerGraph as tg
import json
import pandas as pd

# Connection parameters
hostName = "https://language.i.tgcloud.io/"
userName = "tigergraph"
password = "tigergraph"

conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password)

print("Connected")

Connected


In [40]:
##Clear graph if already solution has the same graph

# conn.gsql('''
# USE GLOBAL
# DROP ALL
# ''')

## Define and Publish the Schema

- Include the edges for the relation types included: **['Antonym','Synonym','RelatedTo']** to generate the gsql create edge query. 
- This will eliminate the effort to type in queries if there are too many relation types.

In [41]:
rel_type = pd.read_csv(root+'encoded/type_rel.csv')
rel = rel_type['type']
print(len(rel))

rel_included = rel #['Antonym','Synonym','RelatedTo']

str_=""
# for r in rel:
for r in rel_included:
  if '/' in r:
    r = r.replace('/','_')
  str_+="CREATE DIRECTED EDGE "+r+"(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE=\"reverse_"+r+"\""

  str_+="\n"
print(str_)

41
CREATE DIRECTED EDGE Attribute(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Attribute"
CREATE DIRECTED EDGE InSynset(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_InSynset"
CREATE DIRECTED EDGE Domain(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Domain"
CREATE DIRECTED EDGE SimilarTo(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_SimilarTo"
CREATE DIRECTED EDGE AlsoSee(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_AlsoSee"
CREATE DIRECTED EDGE IsA(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_IsA"
CREATE DIRECTED EDGE PartOf(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM

- Copy & paste the above query within USE Global gsql block after EDGE:is_pos.

In [42]:
# DEFINE / CREATE ALL EDGES AND VERTICES 
results = conn.gsql('''
USE GLOBAL
CREATE VERTEX synset(PRIMARY_ID id STRING, definition STRING, pos STRING,name STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX word(PRIMARY_ID id STRING, name STRING, pos STRING, conceptUri STRING, label STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX part_of_speech(PRIMARY_ID id STRING, type STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE DIRECTED EDGE has_pos(FROM synset, TO part_of_speech|FROM word, TO part_of_speech) WITH REVERSE_EDGE="reverse_has_pos"
CREATE DIRECTED EDGE is_pos_of(FROM part_of_speech, TO synset|FROM part_of_speech, TO word) WITH REVERSE_EDGE="reverse_is_pos_of"


CREATE DIRECTED EDGE Attribute(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Attribute"
CREATE DIRECTED EDGE InSynset(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_InSynset"
CREATE DIRECTED EDGE Domain(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Domain"
CREATE DIRECTED EDGE SimilarTo(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_SimilarTo"
CREATE DIRECTED EDGE AlsoSee(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_AlsoSee"
CREATE DIRECTED EDGE IsA(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_IsA"
CREATE DIRECTED EDGE PartOf(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_PartOf"
CREATE DIRECTED EDGE Entailment(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Entailment"
CREATE DIRECTED EDGE VerbGroup(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_VerbGroup"
CREATE DIRECTED EDGE Cause(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Cause"
CREATE DIRECTED EDGE Antonym(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Antonym"
CREATE DIRECTED EDGE AtLocation(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_AtLocation"
CREATE DIRECTED EDGE CapableOf(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_CapableOf"
CREATE DIRECTED EDGE Causes(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Causes"
CREATE DIRECTED EDGE CausesDesire(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_CausesDesire"
CREATE DIRECTED EDGE CreatedBy(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_CreatedBy"
CREATE DIRECTED EDGE DefinedAs(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_DefinedAs"
CREATE DIRECTED EDGE DerivedFrom(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_DerivedFrom"
CREATE DIRECTED EDGE Desires(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Desires"
CREATE DIRECTED EDGE DistinctFrom(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_DistinctFrom"
CREATE DIRECTED EDGE Entails(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Entails"
CREATE DIRECTED EDGE EtymologicallyDerivedFrom(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_EtymologicallyDerivedFrom"
CREATE DIRECTED EDGE EtymologicallyRelatedTo(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_EtymologicallyRelatedTo"
CREATE DIRECTED EDGE FormOf(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_FormOf"
CREATE DIRECTED EDGE HasA(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_HasA"
CREATE DIRECTED EDGE HasContext(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_HasContext"
CREATE DIRECTED EDGE HasFirstSubevent(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_HasFirstSubevent"
CREATE DIRECTED EDGE HasLastSubevent(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_HasLastSubevent"
CREATE DIRECTED EDGE HasPrerequisite(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_HasPrerequisite"
CREATE DIRECTED EDGE HasProperty(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_HasProperty"
CREATE DIRECTED EDGE HasSubevent(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_HasSubevent"
CREATE DIRECTED EDGE InstanceOf(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_InstanceOf"
CREATE DIRECTED EDGE LocatedNear(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_LocatedNear"
CREATE DIRECTED EDGE MadeOf(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_MadeOf"
CREATE DIRECTED EDGE MannerOf(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_MannerOf"
CREATE DIRECTED EDGE MotivatedByGoal(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_MotivatedByGoal"
CREATE DIRECTED EDGE NotCapableOf(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_NotCapableOf"
CREATE DIRECTED EDGE NotDesires(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_NotDesires"
CREATE DIRECTED EDGE NotHasProperty(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_NotHasProperty"
CREATE DIRECTED EDGE ReceivesAction(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_ReceivesAction"
CREATE DIRECTED EDGE RelatedTo(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_RelatedTo"
CREATE DIRECTED EDGE SymbolOf(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_SymbolOf"
CREATE DIRECTED EDGE Synonym(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Synonym"
CREATE DIRECTED EDGE UsedFor(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_UsedFor"
CREATE DIRECTED EDGE dbpedia_capital(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_dbpedia_capital"
CREATE DIRECTED EDGE dbpedia_field(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_dbpedia_field"
CREATE DIRECTED EDGE dbpedia_genre(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_dbpedia_genre"
CREATE DIRECTED EDGE dbpedia_genus(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_dbpedia_genus"
CREATE DIRECTED EDGE dbpedia_influencedBy(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_dbpedia_influencedBy"
CREATE DIRECTED EDGE dbpedia_knownFor(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_dbpedia_knownFor"
CREATE DIRECTED EDGE dbpedia_language(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_dbpedia_language"
CREATE DIRECTED EDGE dbpedia_leader(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_dbpedia_leader"
CREATE DIRECTED EDGE dbpedia_occupation(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_dbpedia_occupation"
CREATE DIRECTED EDGE dbpedia_product(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_dbpedia_product"

  ''')
print(results)

Successfully created vertex types: [synset].
Successfully created vertex types: [word].
Successfully created vertex types: [part_of_speech].
Successfully created edge types: [has_pos].
Successfully created reverse edge types: [reverse_has_pos].
Successfully created edge types: [is_pos_of].
Successfully created reverse edge types: [reverse_is_pos_of].
Successfully created edge types: [Attribute].
Successfully created reverse edge types: [reverse_Attribute].
Successfully created edge types: [InSynset].
Successfully created reverse edge types: [reverse_InSynset].
Successfully created edge types: [Domain].
Successfully created reverse edge types: [reverse_Domain].
Successfully created edge types: [SimilarTo].
Successfully created reverse edge types: [reverse_SimilarTo].
Successfully created edge types: [AlsoSee].
Successfully created reverse edge types: [reverse_AlsoSee].
Successfully created edge types: [IsA].
Successfully created reverse edge types: [reverse_IsA].
Successfully created ed

## ConceptNet

## Create ConceptNet Graph

- generate the edges for the relation types included: **['Antonym','Synonym','RelatedTo']** to be included in the graph.

In [43]:
rel = rel_type['type']
print(len(rel))
str_=""
# for r in rel:
for r in rel_included:
  if '/' in r:
    r=r.replace('/','_')
  str_+=r+"\n"
print(str_.strip('\n'))

41
Attribute
InSynset
Domain
SimilarTo
AlsoSee
IsA
PartOf
Entailment
VerbGroup
Cause
Antonym
AtLocation
CapableOf
Causes
CausesDesire
CreatedBy
DefinedAs
DerivedFrom
Desires
DistinctFrom
Entails
EtymologicallyDerivedFrom
EtymologicallyRelatedTo
FormOf
HasA
HasContext
HasFirstSubevent
HasLastSubevent
HasPrerequisite
HasProperty
HasSubevent
InstanceOf
LocatedNear
MadeOf
MannerOf
MotivatedByGoal
NotCapableOf
NotDesires
NotHasProperty
ReceivesAction
RelatedTo


- Copy & paste the above after EDGE: is_pos_of.

In [44]:
results = conn.gsql('CREATE GRAPH ConceptNet(synset, word, part_of_speech,has_pos,is_pos_of,Attribute,InSynset,Domain,SimilarTo,AlsoSee,IsA,PartOf,Entailment,VerbGroup,Cause,Antonym,AtLocation,CapableOf,Causes,CausesDesire,CreatedBy,DefinedAs,DerivedFrom,Desires,DistinctFrom,Entails,EtymologicallyDerivedFrom,EtymologicallyRelatedTo,FormOf,HasA,HasContext,HasFirstSubevent,HasLastSubevent,HasPrerequisite,HasProperty,HasSubevent,InstanceOf,LocatedNear,MadeOf,MannerOf,MotivatedByGoal,NotCapableOf,NotDesires,NotHasProperty,ReceivesAction,RelatedTo,SymbolOf,Synonym,UsedFor,dbpedia_capital,dbpedia_field,dbpedia_genre,dbpedia_genus,dbpedia_influencedBy,dbpedia_knownFor,dbpedia_language,dbpedia_leader,dbpedia_occupation,dbpedia_product)')
print(results)

Stopping GPE GSE RESTPP
Successfully stopped GPE GSE RESTPP in 30.402 seconds
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.026 seconds
The graph ConceptNet is created.


- Generate AuthToken to update the graph

In [45]:
conn.graphname="ConceptNet"
secret = conn.createSecret()
print(secret)
authToken = conn.getToken(secret)
authToken = authToken[0]
print(authToken)
# authToken = 'rc7reopbis1667ksgcppq5v5fb99p6s1'
conn = tg.TigerGraphConnection(host=hostName, graphname="ConceptNet", username=userName, password=password, apiToken=authToken)

def pprint(string):
  print(json.dumps(string, indent=2))

nv234ujgg0mvecam65mqmte1ndfmipcv
5oc81evon5ftmjsbaai2ar6jtvidt6ig


## Create Loading Jobs

- Generate edge queries for the relation types included: **['Antonym','Synonym','RelatedTo']** to include in the load job gsql.

- Here it's important to note that the `$0`, `$1` values line up with the columns of your data.
In this example:
  - `$0` is the `uri` column,
  - `$1` is `id`,
  - `$2` is `word`
  - and so on

In [46]:
rel_type = pd.read_csv(root+'encoded/type_rel.csv')
rel = rel_type['type']


str_=""
# for r in rel:
for r in rel_included:
  orig=r
  if '/' in r:
    r = r.replace('/','_')
  str_+="LOAD MyDataSource TO EDGE  "+r+" VALUES($0 synset, $1 synset) WHERE $4 == \""+orig+"\" USING SEPARATOR=\",\""", HEADER=\"true\", EOL=""\"\\n"""
  str_+="\";\n"
  str_+="LOAD MyDataSource TO EDGE  "+r+" VALUES($0 synset, $1 word) WHERE $4 == \""+orig+"\" USING SEPARATOR=\",\""", HEADER=\"true\", EOL=""\"\\n"""
  str_+="\";\n"
  str_+="LOAD MyDataSource TO EDGE  "+r+" VALUES($0 word, $1 synset) WHERE $4 == \""+orig+"\" USING SEPARATOR=\",\""", HEADER=\"true\", EOL=""\"\\n"""
  str_+="\";\n"
  str_+="LOAD MyDataSource TO EDGE  "+r+" VALUES($0 word, $1 word) WHERE $4 == \""+orig+"\" USING SEPARATOR=\",\""", HEADER=\"true\", EOL=""\"\\n"""
  str_+="\";\n"
print(str_)

LOAD MyDataSource TO EDGE  Attribute VALUES($0 synset, $1 synset) WHERE $4 == "Attribute" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Attribute VALUES($0 synset, $1 word) WHERE $4 == "Attribute" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Attribute VALUES($0 word, $1 synset) WHERE $4 == "Attribute" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Attribute VALUES($0 word, $1 word) WHERE $4 == "Attribute" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  InSynset VALUES($0 synset, $1 synset) WHERE $4 == "InSynset" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  InSynset VALUES($0 synset, $1 word) WHERE $4 == "InSynset" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  InSynset VALUES($0 word, $1 synset) WHERE $4 == "InSynset" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  InSynset VALUES($0 word, $1 word) WHERE

- Copy & Paste the above inside load_job_relationships gsql block.

In [47]:
results = conn.gsql('''
  USE GRAPH ConceptNet
  BEGIN

  CREATE LOADING JOB load_job_relationships FOR GRAPH ConceptNet {
        DEFINE FILENAME MyDataSource;

LOAD MyDataSource TO EDGE  Attribute VALUES($0 synset, $1 synset) WHERE $4 == "Attribute" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Attribute VALUES($0 synset, $1 word) WHERE $4 == "Attribute" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Attribute VALUES($0 word, $1 synset) WHERE $4 == "Attribute" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Attribute VALUES($0 word, $1 word) WHERE $4 == "Attribute" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  InSynset VALUES($0 synset, $1 synset) WHERE $4 == "InSynset" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  InSynset VALUES($0 synset, $1 word) WHERE $4 == "InSynset" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  InSynset VALUES($0 word, $1 synset) WHERE $4 == "InSynset" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  InSynset VALUES($0 word, $1 word) WHERE $4 == "InSynset" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Domain VALUES($0 synset, $1 synset) WHERE $4 == "Domain" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Domain VALUES($0 synset, $1 word) WHERE $4 == "Domain" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Domain VALUES($0 word, $1 synset) WHERE $4 == "Domain" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Domain VALUES($0 word, $1 word) WHERE $4 == "Domain" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  SimilarTo VALUES($0 synset, $1 synset) WHERE $4 == "SimilarTo" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  SimilarTo VALUES($0 synset, $1 word) WHERE $4 == "SimilarTo" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  SimilarTo VALUES($0 word, $1 synset) WHERE $4 == "SimilarTo" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  SimilarTo VALUES($0 word, $1 word) WHERE $4 == "SimilarTo" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  AlsoSee VALUES($0 synset, $1 synset) WHERE $4 == "AlsoSee" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  AlsoSee VALUES($0 synset, $1 word) WHERE $4 == "AlsoSee" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  AlsoSee VALUES($0 word, $1 synset) WHERE $4 == "AlsoSee" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  AlsoSee VALUES($0 word, $1 word) WHERE $4 == "AlsoSee" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  IsA VALUES($0 synset, $1 synset) WHERE $4 == "IsA" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  IsA VALUES($0 synset, $1 word) WHERE $4 == "IsA" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  IsA VALUES($0 word, $1 synset) WHERE $4 == "IsA" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  IsA VALUES($0 word, $1 word) WHERE $4 == "IsA" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  PartOf VALUES($0 synset, $1 synset) WHERE $4 == "PartOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  PartOf VALUES($0 synset, $1 word) WHERE $4 == "PartOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  PartOf VALUES($0 word, $1 synset) WHERE $4 == "PartOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  PartOf VALUES($0 word, $1 word) WHERE $4 == "PartOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Entailment VALUES($0 synset, $1 synset) WHERE $4 == "Entailment" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Entailment VALUES($0 synset, $1 word) WHERE $4 == "Entailment" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Entailment VALUES($0 word, $1 synset) WHERE $4 == "Entailment" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Entailment VALUES($0 word, $1 word) WHERE $4 == "Entailment" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  VerbGroup VALUES($0 synset, $1 synset) WHERE $4 == "VerbGroup" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  VerbGroup VALUES($0 synset, $1 word) WHERE $4 == "VerbGroup" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  VerbGroup VALUES($0 word, $1 synset) WHERE $4 == "VerbGroup" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  VerbGroup VALUES($0 word, $1 word) WHERE $4 == "VerbGroup" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Cause VALUES($0 synset, $1 synset) WHERE $4 == "Cause" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Cause VALUES($0 synset, $1 word) WHERE $4 == "Cause" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Cause VALUES($0 word, $1 synset) WHERE $4 == "Cause" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Cause VALUES($0 word, $1 word) WHERE $4 == "Cause" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Antonym VALUES($0 synset, $1 synset) WHERE $4 == "Antonym" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Antonym VALUES($0 synset, $1 word) WHERE $4 == "Antonym" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Antonym VALUES($0 word, $1 synset) WHERE $4 == "Antonym" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Antonym VALUES($0 word, $1 word) WHERE $4 == "Antonym" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  AtLocation VALUES($0 synset, $1 synset) WHERE $4 == "AtLocation" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  AtLocation VALUES($0 synset, $1 word) WHERE $4 == "AtLocation" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  AtLocation VALUES($0 word, $1 synset) WHERE $4 == "AtLocation" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  AtLocation VALUES($0 word, $1 word) WHERE $4 == "AtLocation" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  CapableOf VALUES($0 synset, $1 synset) WHERE $4 == "CapableOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  CapableOf VALUES($0 synset, $1 word) WHERE $4 == "CapableOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  CapableOf VALUES($0 word, $1 synset) WHERE $4 == "CapableOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  CapableOf VALUES($0 word, $1 word) WHERE $4 == "CapableOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Causes VALUES($0 synset, $1 synset) WHERE $4 == "Causes" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Causes VALUES($0 synset, $1 word) WHERE $4 == "Causes" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Causes VALUES($0 word, $1 synset) WHERE $4 == "Causes" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Causes VALUES($0 word, $1 word) WHERE $4 == "Causes" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  CausesDesire VALUES($0 synset, $1 synset) WHERE $4 == "CausesDesire" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  CausesDesire VALUES($0 synset, $1 word) WHERE $4 == "CausesDesire" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  CausesDesire VALUES($0 word, $1 synset) WHERE $4 == "CausesDesire" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  CausesDesire VALUES($0 word, $1 word) WHERE $4 == "CausesDesire" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  CreatedBy VALUES($0 synset, $1 synset) WHERE $4 == "CreatedBy" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  CreatedBy VALUES($0 synset, $1 word) WHERE $4 == "CreatedBy" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  CreatedBy VALUES($0 word, $1 synset) WHERE $4 == "CreatedBy" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  CreatedBy VALUES($0 word, $1 word) WHERE $4 == "CreatedBy" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  DefinedAs VALUES($0 synset, $1 synset) WHERE $4 == "DefinedAs" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  DefinedAs VALUES($0 synset, $1 word) WHERE $4 == "DefinedAs" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  DefinedAs VALUES($0 word, $1 synset) WHERE $4 == "DefinedAs" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  DefinedAs VALUES($0 word, $1 word) WHERE $4 == "DefinedAs" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  DerivedFrom VALUES($0 synset, $1 synset) WHERE $4 == "DerivedFrom" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  DerivedFrom VALUES($0 synset, $1 word) WHERE $4 == "DerivedFrom" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  DerivedFrom VALUES($0 word, $1 synset) WHERE $4 == "DerivedFrom" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  DerivedFrom VALUES($0 word, $1 word) WHERE $4 == "DerivedFrom" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Desires VALUES($0 synset, $1 synset) WHERE $4 == "Desires" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Desires VALUES($0 synset, $1 word) WHERE $4 == "Desires" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Desires VALUES($0 word, $1 synset) WHERE $4 == "Desires" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Desires VALUES($0 word, $1 word) WHERE $4 == "Desires" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  DistinctFrom VALUES($0 synset, $1 synset) WHERE $4 == "DistinctFrom" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  DistinctFrom VALUES($0 synset, $1 word) WHERE $4 == "DistinctFrom" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  DistinctFrom VALUES($0 word, $1 synset) WHERE $4 == "DistinctFrom" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  DistinctFrom VALUES($0 word, $1 word) WHERE $4 == "DistinctFrom" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Entails VALUES($0 synset, $1 synset) WHERE $4 == "Entails" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Entails VALUES($0 synset, $1 word) WHERE $4 == "Entails" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Entails VALUES($0 word, $1 synset) WHERE $4 == "Entails" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Entails VALUES($0 word, $1 word) WHERE $4 == "Entails" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  EtymologicallyDerivedFrom VALUES($0 synset, $1 synset) WHERE $4 == "EtymologicallyDerivedFrom" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  EtymologicallyDerivedFrom VALUES($0 synset, $1 word) WHERE $4 == "EtymologicallyDerivedFrom" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  EtymologicallyDerivedFrom VALUES($0 word, $1 synset) WHERE $4 == "EtymologicallyDerivedFrom" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  EtymologicallyDerivedFrom VALUES($0 word, $1 word) WHERE $4 == "EtymologicallyDerivedFrom" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  EtymologicallyRelatedTo VALUES($0 synset, $1 synset) WHERE $4 == "EtymologicallyRelatedTo" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  EtymologicallyRelatedTo VALUES($0 synset, $1 word) WHERE $4 == "EtymologicallyRelatedTo" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  EtymologicallyRelatedTo VALUES($0 word, $1 synset) WHERE $4 == "EtymologicallyRelatedTo" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  EtymologicallyRelatedTo VALUES($0 word, $1 word) WHERE $4 == "EtymologicallyRelatedTo" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  FormOf VALUES($0 synset, $1 synset) WHERE $4 == "FormOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  FormOf VALUES($0 synset, $1 word) WHERE $4 == "FormOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  FormOf VALUES($0 word, $1 synset) WHERE $4 == "FormOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  FormOf VALUES($0 word, $1 word) WHERE $4 == "FormOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasA VALUES($0 synset, $1 synset) WHERE $4 == "HasA" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasA VALUES($0 synset, $1 word) WHERE $4 == "HasA" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasA VALUES($0 word, $1 synset) WHERE $4 == "HasA" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasA VALUES($0 word, $1 word) WHERE $4 == "HasA" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasContext VALUES($0 synset, $1 synset) WHERE $4 == "HasContext" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasContext VALUES($0 synset, $1 word) WHERE $4 == "HasContext" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasContext VALUES($0 word, $1 synset) WHERE $4 == "HasContext" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasContext VALUES($0 word, $1 word) WHERE $4 == "HasContext" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasFirstSubevent VALUES($0 synset, $1 synset) WHERE $4 == "HasFirstSubevent" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasFirstSubevent VALUES($0 synset, $1 word) WHERE $4 == "HasFirstSubevent" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasFirstSubevent VALUES($0 word, $1 synset) WHERE $4 == "HasFirstSubevent" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasFirstSubevent VALUES($0 word, $1 word) WHERE $4 == "HasFirstSubevent" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasLastSubevent VALUES($0 synset, $1 synset) WHERE $4 == "HasLastSubevent" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasLastSubevent VALUES($0 synset, $1 word) WHERE $4 == "HasLastSubevent" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasLastSubevent VALUES($0 word, $1 synset) WHERE $4 == "HasLastSubevent" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasLastSubevent VALUES($0 word, $1 word) WHERE $4 == "HasLastSubevent" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasPrerequisite VALUES($0 synset, $1 synset) WHERE $4 == "HasPrerequisite" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasPrerequisite VALUES($0 synset, $1 word) WHERE $4 == "HasPrerequisite" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasPrerequisite VALUES($0 word, $1 synset) WHERE $4 == "HasPrerequisite" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasPrerequisite VALUES($0 word, $1 word) WHERE $4 == "HasPrerequisite" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasProperty VALUES($0 synset, $1 synset) WHERE $4 == "HasProperty" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasProperty VALUES($0 synset, $1 word) WHERE $4 == "HasProperty" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasProperty VALUES($0 word, $1 synset) WHERE $4 == "HasProperty" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasProperty VALUES($0 word, $1 word) WHERE $4 == "HasProperty" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasSubevent VALUES($0 synset, $1 synset) WHERE $4 == "HasSubevent" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasSubevent VALUES($0 synset, $1 word) WHERE $4 == "HasSubevent" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasSubevent VALUES($0 word, $1 synset) WHERE $4 == "HasSubevent" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  HasSubevent VALUES($0 word, $1 word) WHERE $4 == "HasSubevent" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  InstanceOf VALUES($0 synset, $1 synset) WHERE $4 == "InstanceOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  InstanceOf VALUES($0 synset, $1 word) WHERE $4 == "InstanceOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  InstanceOf VALUES($0 word, $1 synset) WHERE $4 == "InstanceOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  InstanceOf VALUES($0 word, $1 word) WHERE $4 == "InstanceOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  LocatedNear VALUES($0 synset, $1 synset) WHERE $4 == "LocatedNear" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  LocatedNear VALUES($0 synset, $1 word) WHERE $4 == "LocatedNear" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  LocatedNear VALUES($0 word, $1 synset) WHERE $4 == "LocatedNear" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  LocatedNear VALUES($0 word, $1 word) WHERE $4 == "LocatedNear" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  MadeOf VALUES($0 synset, $1 synset) WHERE $4 == "MadeOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  MadeOf VALUES($0 synset, $1 word) WHERE $4 == "MadeOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  MadeOf VALUES($0 word, $1 synset) WHERE $4 == "MadeOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  MadeOf VALUES($0 word, $1 word) WHERE $4 == "MadeOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  MannerOf VALUES($0 synset, $1 synset) WHERE $4 == "MannerOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  MannerOf VALUES($0 synset, $1 word) WHERE $4 == "MannerOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  MannerOf VALUES($0 word, $1 synset) WHERE $4 == "MannerOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  MannerOf VALUES($0 word, $1 word) WHERE $4 == "MannerOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  MotivatedByGoal VALUES($0 synset, $1 synset) WHERE $4 == "MotivatedByGoal" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  MotivatedByGoal VALUES($0 synset, $1 word) WHERE $4 == "MotivatedByGoal" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  MotivatedByGoal VALUES($0 word, $1 synset) WHERE $4 == "MotivatedByGoal" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  MotivatedByGoal VALUES($0 word, $1 word) WHERE $4 == "MotivatedByGoal" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  NotCapableOf VALUES($0 synset, $1 synset) WHERE $4 == "NotCapableOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  NotCapableOf VALUES($0 synset, $1 word) WHERE $4 == "NotCapableOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  NotCapableOf VALUES($0 word, $1 synset) WHERE $4 == "NotCapableOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  NotCapableOf VALUES($0 word, $1 word) WHERE $4 == "NotCapableOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  NotDesires VALUES($0 synset, $1 synset) WHERE $4 == "NotDesires" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  NotDesires VALUES($0 synset, $1 word) WHERE $4 == "NotDesires" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  NotDesires VALUES($0 word, $1 synset) WHERE $4 == "NotDesires" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  NotDesires VALUES($0 word, $1 word) WHERE $4 == "NotDesires" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  NotHasProperty VALUES($0 synset, $1 synset) WHERE $4 == "NotHasProperty" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  NotHasProperty VALUES($0 synset, $1 word) WHERE $4 == "NotHasProperty" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  NotHasProperty VALUES($0 word, $1 synset) WHERE $4 == "NotHasProperty" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  NotHasProperty VALUES($0 word, $1 word) WHERE $4 == "NotHasProperty" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  ReceivesAction VALUES($0 synset, $1 synset) WHERE $4 == "ReceivesAction" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  ReceivesAction VALUES($0 synset, $1 word) WHERE $4 == "ReceivesAction" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  ReceivesAction VALUES($0 word, $1 synset) WHERE $4 == "ReceivesAction" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  ReceivesAction VALUES($0 word, $1 word) WHERE $4 == "ReceivesAction" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  RelatedTo VALUES($0 synset, $1 synset) WHERE $4 == "RelatedTo" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  RelatedTo VALUES($0 synset, $1 word) WHERE $4 == "RelatedTo" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  RelatedTo VALUES($0 word, $1 synset) WHERE $4 == "RelatedTo" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  RelatedTo VALUES($0 word, $1 word) WHERE $4 == "RelatedTo" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  SymbolOf VALUES($0 synset, $1 synset) WHERE $4 == "SymbolOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  SymbolOf VALUES($0 synset, $1 word) WHERE $4 == "SymbolOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  SymbolOf VALUES($0 word, $1 synset) WHERE $4 == "SymbolOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  SymbolOf VALUES($0 word, $1 word) WHERE $4 == "SymbolOf" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Synonym VALUES($0 synset, $1 synset) WHERE $4 == "Synonym" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Synonym VALUES($0 synset, $1 word) WHERE $4 == "Synonym" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Synonym VALUES($0 word, $1 synset) WHERE $4 == "Synonym" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  Synonym VALUES($0 word, $1 word) WHERE $4 == "Synonym" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  UsedFor VALUES($0 synset, $1 synset) WHERE $4 == "UsedFor" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  UsedFor VALUES($0 synset, $1 word) WHERE $4 == "UsedFor" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  UsedFor VALUES($0 word, $1 synset) WHERE $4 == "UsedFor" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  UsedFor VALUES($0 word, $1 word) WHERE $4 == "UsedFor" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_capital VALUES($0 synset, $1 synset) WHERE $4 == "dbpedia/capital" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_capital VALUES($0 synset, $1 word) WHERE $4 == "dbpedia/capital" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_capital VALUES($0 word, $1 synset) WHERE $4 == "dbpedia/capital" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_capital VALUES($0 word, $1 word) WHERE $4 == "dbpedia/capital" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_field VALUES($0 synset, $1 synset) WHERE $4 == "dbpedia/field" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_field VALUES($0 synset, $1 word) WHERE $4 == "dbpedia/field" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_field VALUES($0 word, $1 synset) WHERE $4 == "dbpedia/field" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_field VALUES($0 word, $1 word) WHERE $4 == "dbpedia/field" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_genre VALUES($0 synset, $1 synset) WHERE $4 == "dbpedia/genre" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_genre VALUES($0 synset, $1 word) WHERE $4 == "dbpedia/genre" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_genre VALUES($0 word, $1 synset) WHERE $4 == "dbpedia/genre" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_genre VALUES($0 word, $1 word) WHERE $4 == "dbpedia/genre" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_genus VALUES($0 synset, $1 synset) WHERE $4 == "dbpedia/genus" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_genus VALUES($0 synset, $1 word) WHERE $4 == "dbpedia/genus" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_genus VALUES($0 word, $1 synset) WHERE $4 == "dbpedia/genus" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_genus VALUES($0 word, $1 word) WHERE $4 == "dbpedia/genus" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_influencedBy VALUES($0 synset, $1 synset) WHERE $4 == "dbpedia/influencedBy" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_influencedBy VALUES($0 synset, $1 word) WHERE $4 == "dbpedia/influencedBy" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_influencedBy VALUES($0 word, $1 synset) WHERE $4 == "dbpedia/influencedBy" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_influencedBy VALUES($0 word, $1 word) WHERE $4 == "dbpedia/influencedBy" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_knownFor VALUES($0 synset, $1 synset) WHERE $4 == "dbpedia/knownFor" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_knownFor VALUES($0 synset, $1 word) WHERE $4 == "dbpedia/knownFor" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_knownFor VALUES($0 word, $1 synset) WHERE $4 == "dbpedia/knownFor" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_knownFor VALUES($0 word, $1 word) WHERE $4 == "dbpedia/knownFor" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_language VALUES($0 synset, $1 synset) WHERE $4 == "dbpedia/language" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_language VALUES($0 synset, $1 word) WHERE $4 == "dbpedia/language" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_language VALUES($0 word, $1 synset) WHERE $4 == "dbpedia/language" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_language VALUES($0 word, $1 word) WHERE $4 == "dbpedia/language" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_leader VALUES($0 synset, $1 synset) WHERE $4 == "dbpedia/leader" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_leader VALUES($0 synset, $1 word) WHERE $4 == "dbpedia/leader" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_leader VALUES($0 word, $1 synset) WHERE $4 == "dbpedia/leader" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_leader VALUES($0 word, $1 word) WHERE $4 == "dbpedia/leader" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_occupation VALUES($0 synset, $1 synset) WHERE $4 == "dbpedia/occupation" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_occupation VALUES($0 synset, $1 word) WHERE $4 == "dbpedia/occupation" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_occupation VALUES($0 word, $1 synset) WHERE $4 == "dbpedia/occupation" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_occupation VALUES($0 word, $1 word) WHERE $4 == "dbpedia/occupation" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_product VALUES($0 synset, $1 synset) WHERE $4 == "dbpedia/product" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_product VALUES($0 synset, $1 word) WHERE $4 == "dbpedia/product" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_product VALUES($0 word, $1 synset) WHERE $4 == "dbpedia/product" USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD MyDataSource TO EDGE  dbpedia_product VALUES($0 word, $1 word) WHERE $4 == "dbpedia/product" USING SEPARATOR=",", HEADER="true", EOL="\n";
      }


  CREATE LOADING JOB load_job_pos_wn FOR GRAPH ConceptNet {
        DEFINE FILENAME MyDataSource;
        LOAD MyDataSource TO VERTEX part_of_speech VALUES($2, $1) USING SEPARATOR=",", HEADER="true", EOL="\n";
      }


  CREATE LOADING JOB load_job_synsets_encoded FOR GRAPH ConceptNet {
        DEFINE FILENAME MyDataSource;
        LOAD MyDataSource TO VERTEX synset VALUES($0, $2, $1, $4) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      }

  CREATE LOADING JOB load_job_words_encoded FOR GRAPH ConceptNet {
        DEFINE FILENAME MyDataSource;
        LOAD MyDataSource TO VERTEX word VALUES($0, $1, $2, $3, $4) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
        LOAD MyDataSource TO EDGE has_pos VALUES($0 word, $2 part_of_speech) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
        LOAD MyDataSource TO EDGE is_pos_of VALUES($2 part_of_speech, $0 word) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
        LOAD MyDataSource TO EDGE has_pos VALUES($0 synset, $2 part_of_speech) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
        LOAD MyDataSource TO EDGE is_pos_of VALUES($2 part_of_speech, $0 synset) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      }


  END
  ''')
print(results)

Using graph 'ConceptNet'
Successfully created loading jobs: [load_job_relationships].
Successfully created loading jobs: [load_job_pos_wn].
Successfully created loading jobs: [load_job_synsets_encoded].
Successfully created loading jobs: [load_job_words_encoded].


## Load Data

### load relationships

In [48]:
# Load the posts file wiht the 'load_words' job
load_words = root+'relationships.csv'
results = conn.uploadFile(load_words, timeout=1000000,fileTag='MyDataSource', jobName='load_job_relationships')
# print(json.dumps(results, indent=2))

### load pos

In [49]:
# Load the posts file wiht the 'load_words' job
load_words = root+'encoded/pos_wn.csv'
results = conn.uploadFile(load_words, timeout=100000,fileTag='MyDataSource', jobName='load_job_pos_wn')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 6,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "part_of_speech",
          "validObject": 6,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


### load words and synsets

In [51]:
# Load the posts file wiht the 'load_edges' job
load_edges = root+'encoded/synsets-encoded.csv'
results = conn.uploadFile(load_edges, timeout=100000, fileTag='MyDataSource', jobName='load_job_synsets_encoded')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 117660,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "synset",
          "validObject": 117659,
          "noIdFound": 1,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


In [52]:
# Load the posts file wiht the 'load_edges' job
load_edges = root+'encoded/words_encoded.csv'
results = conn.uploadFile(load_edges, timeout=100000, fileTag='MyDataSource', jobName='load_job_words_encoded')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 1530137,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "word",
          "validObject": 1530135,
          "noIdFound": 2,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [
        {
          "typeName": "has_pos",
          "validObject": 1530135,
          "noIdFound": 2,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        },
        {
          "typeName": "has_pos",
          "validObject": 1530135,
          "noIdFound": 2,
          "invalidAttribute": 0,
          "invalidVe

## Exploring the Graph

### Get Vertex and Edge Schema

In [53]:
results = conn.getVertexTypes()
print(f"Verticies: {results}")
vertices = results

results = conn.getEdgeTypes()
print(f"Edges: {results}")
edges = results

Verticies: ['synset', 'word', 'part_of_speech']
Edges: ['has_pos', 'is_pos_of', 'Attribute', 'InSynset', 'Domain', 'SimilarTo', 'AlsoSee', 'IsA', 'PartOf', 'Entailment', 'VerbGroup', 'Cause', 'Antonym', 'AtLocation', 'CapableOf', 'Causes', 'CausesDesire', 'CreatedBy', 'DefinedAs', 'DerivedFrom', 'Desires', 'DistinctFrom', 'Entails', 'EtymologicallyDerivedFrom', 'EtymologicallyRelatedTo', 'FormOf', 'HasA', 'HasContext', 'HasFirstSubevent', 'HasLastSubevent', 'HasPrerequisite', 'HasProperty', 'HasSubevent', 'InstanceOf', 'LocatedNear', 'MadeOf', 'MannerOf', 'MotivatedByGoal', 'NotCapableOf', 'NotDesires', 'NotHasProperty', 'ReceivesAction', 'RelatedTo', 'SymbolOf', 'Synonym', 'UsedFor', 'dbpedia_capital', 'dbpedia_field', 'dbpedia_genre', 'dbpedia_genus', 'dbpedia_influencedBy', 'dbpedia_knownFor', 'dbpedia_language', 'dbpedia_leader', 'dbpedia_occupation', 'dbpedia_product']


In [54]:

print(f"Results for Post vertex")
pprint(conn.getVertexType("word"))

print("-----------------")
print(f"Results for liked edge")
pprint(conn.getEdgeType("type"))


Results for Post vertex
{
  "Config": {
    "TAGGABLE": false,
    "STATS": "OUTDEGREE_BY_EDGETYPE",
    "PRIMARY_ID_AS_ATTRIBUTE": true
  },
  "Attributes": [
    {
      "AttributeType": {
        "Name": "STRING"
      },
      "IsPartOfCompositeKey": false,
      "PrimaryIdAsAttribute": false,
      "AttributeName": "name",
      "HasIndex": false,
      "internalAttribute": false,
      "IsPrimaryKey": false
    },
    {
      "AttributeType": {
        "Name": "STRING"
      },
      "IsPartOfCompositeKey": false,
      "PrimaryIdAsAttribute": false,
      "AttributeName": "pos",
      "HasIndex": false,
      "internalAttribute": false,
      "IsPrimaryKey": false
    },
    {
      "AttributeType": {
        "Name": "STRING"
      },
      "IsPartOfCompositeKey": false,
      "PrimaryIdAsAttribute": false,
      "AttributeName": "conceptUri",
      "HasIndex": false,
      "internalAttribute": false,
      "IsPrimaryKey": false
    },
    {
      "AttributeType": {
        "Nam

## Counting Data

In [55]:
print("Vertex Counts")
for vertex in vertices:
  print(f"There are {conn.getVertexCount(vertex)} {vertex} vertices in the graph")

print("--------------")
print("Edge Counts")
for edge in edges:
  print(f"There are {conn.getEdgeCount(edge)} {edge} edges in the graph")

Vertex Counts
There are 1765187 synset vertices in the graph
There are 1647764 word vertices in the graph
There are 8 part_of_speech vertices in the graph
--------------
Edge Counts
There are 1793122 has_pos edges in the graph
There are 3060270 is_pos_of edges in the graph
There are 2556 Attribute edges in the graph
There are 827764 InSynset edges in the graph
There are 35816 Domain edges in the graph
There are 120088 SimilarTo edges in the graph
There are 5788 AlsoSee edges in the graph
There are 1255492 IsA edges in the graph
There are 140344 PartOf edges in the graph
There are 1632 Entailment edges in the graph
There are 3496 VerbGroup edges in the graph
There are 880 Cause edges in the graph
There are 72080 Antonym edges in the graph
There are 110744 AtLocation edges in the graph
There are 90704 CapableOf edges in the graph
There are 67156 Causes edges in the graph
There are 18752 CausesDesire edges in the graph
There are 1048 CreatedBy edges in the graph
There are 8680 DefinedAs e

# Clear the Whole Graph
DANGER ZONE

In [ ]:
# conn.gsql('''
# USE GLOBAL
# DROP ALL
# ''')